# Aprendizaje No Supervisado
### Grupo 23

**Integrantes:** Franco Amilcar Genolet (francogeno97@gmail.com), Fabian Alejandro Zapata Cerutti (fzc501@gmail.com), Luis Alejandro Guedez Gomez (luis.guedez@dicsys.com), María Laura Mantovani (mantovanimlaura@gmail.com).

## Inicialización del entorno

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
from plotly.offline import init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
#clusters
from sklearn.cluster import KMeans,MeanShift
from sklearn import decomposition

## 1. Análisis exploratorio de la base

Descargamos el archivo y vemos las primeras dos líneas:

In [ ]:
#df = pd.read_csv('https://raw.githubusercontent.com/AlejandroGuedez/Equipo-23-de-Diplomado-data-scientist/Versiones/players_22.csv', low_memory=False)

df=pd.read_csv("players_22.csv")
df.head(2)

Revisamos las columnas (variables) del dataset:

In [ ]:
variables = df.columns.values.tolist()
for v in variables:
    print(v)
print(len(df.columns), 'variables')
print(len(df))

A diferencia del dataset revisado en clase, que tenía 89 variables, éste tiene 110. Vemos qué tipo de variables son, y evaluamos la proporción de nulos y ceros:

In [ ]:
def status(data):

    data2=data 
    # total de rows
    tot_rows=len(data2)
    # total de nan
    d2=data2.isnull().sum().reset_index()
    d2.columns=['variable', 'q_nan']
    # percentage of nan
    d2[['p_nan']]=d2[['q_nan']]/tot_rows
    # num of zeros
    d2['q_zeros']=(data2==0).sum().values
    # perc of zeros
    d2['p_zeros']=d2[['q_zeros']]/tot_rows
    # total unique values
    d2['unique']=data2.nunique().values
    # get data types per column
    d2['type']=[str(x) for x in data2.dtypes.values]
    return(d2)
status(df)

Las variables `pace`, `shooting`, `passing`, `dribbling`, `defending` y `physic` tienen 2132 registros nulos. Sospechamos que esto coincide con jugadores cuya única posición es GK (goalkeeper).

In [ ]:
len(df.player_positions[df.player_positions == 'GK'])

In [ ]:
# Datos de jugadores que solo son GK
interesting_cols = [ 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic' ]
gk = df[df.player_positions == 'GK']
gk[interesting_cols]

Confirmamos la sospecha. 

En cuanto a la variable `goalkeeping_speed`, que tiene 17107 valores nulos, ¿será que ésta corresponde a jugadores que nunca juegan como arqueros?

In [ ]:
len(df[~df.player_positions.str.contains('GK')])

In [ ]:
# Datos de jugadores que nunca son GK
interesting_cols = [ 'goalkeeping_speed' ]
non_gk = df[~df.player_positions.str.contains('GK')]
non_gk[interesting_cols].describe()

Confirmamos esta otra sospecha. Si queremos incluir estas variables en el análisis, lo más lógico sería separar inicialmente a los arqueros.

Visualizamos gráficamente los datos faltantes:

In [ ]:
null_values_series = df.isnull().sum().where(lambda x : x > 0).dropna().astype('Int32')
print(null_values_series.sort_values(ascending=False).to_string()) # to_string() removes the name and dtype from the output
msno.matrix(df[null_values_series.index.tolist()], figsize=(15, 8));

Las variables que más datos faltantes tienen son las categóricas que son menos relevantes para este ejercicio. En cuanto a las numéricas (a partir de `physic` y hasta `pace` en el listado anterior), ya se explicó la razón por la cual faltan esos datos. En la gráfica se ve que los datos que faltan para una de estas variables, faltan también para el resto y se debe a la posición del jugador.

Vemos algunas medidas descriptivas para las variables numéricas:

In [ ]:
df.describe() #60 variables numericas

Veamos gráficamente algunas de estas variables numéricas. El gráfico que sigue muestra la relación entre las variables potencial y salario, en función a la reputación internacional del jugador:

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.scatterplot(x =df['potential'], y = df['wage_eur'], hue = df['international_reputation'])
plt.xlabel("Potential") 
plt.ylabel("Wage EUR")
plt.title("Potential & wage", fontsize = 18)
plt.show()

Como es de esperarse, hay una gran correlacion entre el potencial y el salario cobrado en euros, a medida que la reputacion internacional aumenta, mas salario cobra tambien.

Las variables numéricas, tal como se observó en clase, son en su mayoría discretas. A partir de la variable `pace` y hasta `goalkeeping_speed` son variables numéricas discretas que van de 0 a 100. Estas variables son de desempeño del jugador, y las llamaremos skills_ratings. Dado que todas tienen la misma escala, será innecesario escalar como paso previo al clustering.

Para una primera aproximación global a los jugadores, analizaremos la variable `overall`. Como se vió en clase, esta variable se calcula usando otras variables de desempeño del jugador (skills_ratings), utilizando redondeo. Es decir que puede darnos una idea general de la performance global por jugador. Dicha variable es también numérica discreta, y toma valores enteros entre 0 y 100.

Realizamos el histograma de la variable `overall` (desempeño global):

In [ ]:
nbins=1*(df.overall.max()-df.overall.min())+1
df['overall'].hist(bins = nbins)
print(nbins)

Veamos la correlacion entre la valoracion general y el salario:

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
plt.scatter(x=df['potential'], y=df['value_eur'] )
plt.xlabel("Overall") 
plt.ylabel("Value in EUR")
plt.title("Overall & Value in EUR", fontsize = 15)
plt.show()

Como tambien era de esperarse, se observa una marcada correlacion entre la valoracion general y el salario

Vemos la distribución de algunas variables puntuales:

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(13, 9))
axes[0,0].hist(df['wage_eur'])
axes[0,0].set_xlabel('Wages in Euro')
axes[0,0].set_ylabel('Count')
axes[0,0].set_title('Distribution of Wages in Euros')

axes[0,1].hist(df['age'], bins = 15)
axes[0,1].set_xlabel('Age of Players')
axes[0,1].set_ylabel('Count')
axes[0,1].set_title('Distribution of Players Ages')

# first two is using a matplotlib syntax, the next two I'll do with seaborn 

axes[1,0].set_title('Distribution of Height of Players')
sns.histplot(df, x='height_cm', ax=axes[1,0], kde=True)
axes[1,0].set_xlabel('Height in Centimeters')
axes[1,0].set_ylabel('Count')


axes[1,1].set_title('Distribution of Weight of Players')
sns.histplot(df, x='weight_kg', ax=axes[1,1], kde=True)
axes[1,1].set_xlabel('Weight in kg')
axes[1,1].set_ylabel('Count')


plt.tight_layout(pad=2)
plt.show()

Se observa que los salarios en Euros son en su mayoría menores a 50k, que la edad de los jugadores se centra entre 20 y 30 años. La altura y el peso de los jugadores tienen distribuciones acampanadas centradas aproximadamente en los 182 cm y los 75kg. 

Veamos de qué nacionalidades son los jugadores de la base:

In [ ]:
from collections import Counter
bar_plot = dict(Counter(df['nationality_name'].values).most_common(10))
bar_plot

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
plt.bar(*zip(*bar_plot.items()))
ax.set_title('Most Popular Nationalities')
plt.show()

Analicemos ahora el puntaje en función a sus nacionalidades:

In [ ]:
stats_rank = df.groupby(['nationality_name']).mean()
print(stats_rank['overall'].sort_values(ascending=False))

Se observa que los puntajes promedio más altos de los jugadores son de países menos populares según se vió en la gráfica anterior. 

Análogamente a como se hizo en clase, miramos ahora los mejores jugadores por su posición. En este dataset, no hay una única columna de posición, sino que hay dos: una muestra todas las posiciones en las que juega el jugador (`player_positions`, un mismo jugador puede jugar en más de una posición) y la otra muestra la posición que tiene en el club en el que juega. Usaremos esta última para simplificar este análisis: 

In [ ]:
best_players_per_position=df.iloc[df.groupby(df['club_position'])['overall'].idxmax()][['club_position','short_name','overall']]
best_players_per_position

Vemos cuántos jugadores hay por cada posición (tomamos la posición que tienen en el club):

In [ ]:
pd.DataFrame(df.club_position.value_counts().sort_index())

Aquí se observa un valor más chico de arqueros (701). 

Vemos los promedios de la variable `overall` por club (primeros 10):

In [ ]:
club_avg_overall=df.groupby("club_name")["overall"].mean().reset_index().sort_values("overall",ascending=False)
club_avg_overall.head(10)

## 2. Evaluación visual  e intuitiva de a dos variables numéricas por vez

Esperamos que los resultados de los goalkeepers (arqueros/porteros) sean diferentes al resto, por lo que antes de analizar visualmente las variables, definiremos a estos dentro de un grupo separado:

In [ ]:
df['gk_new']=df.player_positions.apply(lambda x: 1 if x == 'GK' else 0)
df[['player_positions','gk_new']].head(10)

Una vez separados los arqueros, armamos el conjunto de skills_ratings:

In [ ]:
skills_ratings = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 
                    'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 
                    'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 
                    'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 
                    'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 
                    'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 
                    'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 
                    'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 
                    'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 
                    'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']

In [ ]:
len(skills_ratings)

Tenemos 41 variables que queremos ver como se comportan entre sí. Con el siguiente código graficaremos estas 41 variables de a pares.

In [ ]:
n_cols = 5
n_rows = int((len(skills_ratings)) // n_cols) + (len(skills_ratings) % n_cols > 0)
x = 0
for i, col in enumerate(skills_ratings):
    subplot = 0
    fig = plt.figure(figsize=(30, 40))
    for x in range(i,len(skills_ratings)):
        subplot = subplot + 1
        ax = fig.add_subplot(n_rows, n_cols, subplot)
        ax.set_title(col + ' vs. ' + skills_ratings[x])
        if x == i:
            sns.kdeplot(data=df, x=skills_ratings[i], fill=True)#, hue='')
        else:
            sns.scatterplot(data=df.sample(5200), x=skills_ratings[i], y=skills_ratings[x], hue='gk_new')
    print('*  '*50)
    print(col)
    fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.show()   
    plt.close()

Lo primero que se observa de las gráficas, es que las primeras variables (`pace`, `shooting`, `passing`, `dribbling`, `defending`, y `physic`) no tienen puntos naranjas, que son los que corresponden a los goalkeepers (arqueros/porteros). Ya habíamos mencionado que dichas variables eran nulas para este grupo. Por la misma razón, para estas variables el gráfico de a pares con `goalkeeping_speed` queda vacío. 

Además, los gráficos de variables contra las cuales si se puede cruzar `goalkeeping_speed` solo tienen puntos naranjas (goalkeepers). 

En las gráficas en las que están tanto goalkeepers con el resto de los jugadores, se observa claramente como los primeros se separan del resto. Cabe pensar que la posición en la cancha aporta un dato relevante para el armado de los clusters.



## 3. Técnicas de clustering

In [ ]:
skills_ratings_non_gk = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 
                    'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 
                    'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 
                    'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 
                    'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 
                    'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 
                    'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 
                    'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 
                    'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 
                    'goalkeeping_positioning', 'goalkeeping_reflexes']

#### 3.1. Kmeans

Debemos definir la cantidad de clusters. Probemos con 2, 3 y 4 clusters.

In [ ]:
non_gk_skills=non_gk[skills_ratings_non_gk]

km_2 = KMeans(n_clusters=2)
km_2.fit(non_gk_skills)
non_gk_skills['kmeans_2'] = km_2.labels_ # Agregamos las etiquetas al df

km_3 = KMeans(n_clusters=3)
km_3.fit(non_gk_skills)
non_gk_skills['kmeans_3'] = km_3.labels_ # Agregamos las etiquetas al df

km_4 = KMeans(n_clusters=4)
km_4.fit(non_gk_skills)
non_gk_skills['kmeans_4'] = km_4.labels_ # Agregamos las etiquetas al df

In [ ]:
non_gk_skills.head(5)

Antes de repetir las gráficas, veamos los puntajes promedio para cada columna por cluster, según las 3 alternativas que probamos (2, 3 y 4 clusters)

**K-means de 2 clusters**

In [ ]:
means_all = non_gk_skills.iloc[:, 0:40].mean()#.map('{:,.0f}'.format)
means_k2 = non_gk_skills.groupby('kmeans_2')[non_gk_skills.columns[0:40]].mean().transpose()
means_k2['total'] = means_all
means_k2['0_vs_total'] = (((means_k2[0] / means_k2['total']) -1)  *100).map('{:,.0f}%'.format)
means_k2['1_vs_total'] = (((means_k2[1] / means_k2['total']) -1)  *100).map('{:,.0f}%'.format)
means_k2[0] = means_k2[0].map('{:,.0f}'.format)
means_k2[1] = means_k2[1].map('{:,.0f}'.format)
means_k2['total'] = means_k2['total'].map('{:,.0f}'.format)
means_k2

In [ ]:
non_gk_skills.kmeans_2.value_counts()

Al dividir los jugadores (excluyendo arqueros) en dos clusters, se observa que el cluster 0 parece ser más fuerte en defensa, ya que para las 3 habilidades de defensa (`defending_marking_awareness`, `defending_standing_tackle` y `defending_sliding_tackle`), el puntaje promedio de los jugadores es casi el doble de los del cluster 1. Por el contrario, el cluster 1 tiene bajo puntaje promedio para dichas habilidades, y en cambio muestra puntajes más altos para las habilidades más relacionadas con el ataque, como `attacking_volleys`, `attacking_fishing`, `shooting`. Ninguno de los clusters muestra diferencias marcadas para las habilidades relacionadas a goalkeeping. Esto tiene sentido, ya que ninguno de estos jugadores es arquero. Probablemente convenga quitar estas variables para el armado de los clusters, ya que no están aportando información significativa.

**K-means de 3 clusters**

In [ ]:
means_k3 = non_gk_skills.groupby('kmeans_3')[non_gk_skills.columns[0:40]].mean().transpose()
means_k3['total'] = means_all
means_k3['0_vs_total'] = (((means_k3[0] / means_k3['total']) -1)  *100).map('{:,.0f}%'.format)
means_k3['1_vs_total'] = (((means_k3[1] / means_k3['total']) -1)  *100).map('{:,.0f}%'.format)
means_k3['2_vs_total'] = (((means_k3[2] / means_k3['total']) -1)  *100).map('{:,.0f}%'.format)
means_k3[0] = means_k3[0].map('{:,.0f}'.format)
means_k3[1] = means_k3[1].map('{:,.0f}'.format)
means_k3[2] = means_k3[2].map('{:,.0f}'.format)
means_k3['total'] = means_k3['total'].map('{:,.0f}'.format)
means_k3

In [ ]:
non_gk_skills.kmeans_3.value_counts()

Al dividir en tres clusters, parece que el que antes era el cluster 1 ahora es el 2, y el que antes era el cluster 0 se dividió en 2. El ahora cluster 2 (antes 1) tiene bajo puntaje promedio para las habilidades de defensa, y muestra puntajes más altos para las habilidades más relacionadas con el ataque, como `attacking_volleys`, `attacking_fishing`, `shooting`. Nuevamente ninguno de los clusters muestra diferencias marcadas para las habilidades relacionadas a goalkeeping. 

**K-means de 4 clusters**

In [ ]:
means_k4 = non_gk_skills.groupby('kmeans_4')[non_gk_skills.columns[0:40]].mean().transpose()
means_k4['total'] = means_all
means_k4['0_vs_total'] = (((means_k4[0] / means_k4['total']) -1)  *100).map('{:,.0f}%'.format)
means_k4['1_vs_total'] = (((means_k4[1] / means_k4['total']) -1)  *100).map('{:,.0f}%'.format)
means_k4['2_vs_total'] = (((means_k4[2] / means_k4['total']) -1)  *100).map('{:,.0f}%'.format)
means_k4['3_vs_total'] = (((means_k4[3] / means_k4['total']) -1)  *100).map('{:,.0f}%'.format)
means_k4[0] = means_k4[0].map('{:,.0f}'.format)
means_k4[1] = means_k4[1].map('{:,.0f}'.format)
means_k4[2] = means_k4[2].map('{:,.0f}'.format)
means_k4[3] = means_k4[3].map('{:,.0f}'.format)
means_k4['total'] = means_k4['total'].map('{:,.0f}'.format)
means_k4

In [ ]:
non_gk_skills.kmeans_4.value_counts()

Al dividir en cuatro clusters, parece que un porcentaje que conformaba antes el cluster 1 ahora está en el 2. Este cluster se caracteriza por tener bajos puntajes en las habilidades relacionadas con el ataque: `shooting`, `passing`, `dribbling`, `attacking_finishing`, `attacking_volleys`, y `skill_dribbling`, y puntajes mas altos que el promedio en las habilidades de defensa: `defending_marking_awareness`, `defending_standing_tackle` y `defending_sliding_tackle`. 

REDACTAR MEJOR: nos parece en terminos d interpretacion, mejor quedarnos con 2 clusters (fuertes en defensa, vs fuertes en ataque)


Quitamos las habilidades de goalkeeping:

In [ ]:
skills_ratings_non_gk_new = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 
                    'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 
                    'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 
                    'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 
                    'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 
                    'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 
                    'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 
                    'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 
                    'defending_sliding_tackle']

Veamos gráficamente como quedan los clusters.

In [ ]:
non_gk_skills_new=non_gk[skills_ratings_non_gk_new]

km_2_new = KMeans(n_clusters=2)
km_2_new.fit(non_gk_skills_new)
non_gk_skills_new['kmeans_2_new'] = km_2_new.labels_ # Agregamos las etiquetas al df

In [ ]:
n_cols = 5
n_rows = int((len(skills_ratings_non_gk_new)) // n_cols) + (len(skills_ratings_non_gk_new) % n_cols > 0)
x = 0
for i, col in enumerate(skills_ratings_non_gk_new):
    subplot = 0
    fig = plt.figure(figsize=(30, 40))
    for x in range(i,len(skills_ratings_non_gk_new)):
        subplot = subplot + 1
        ax = fig.add_subplot(n_rows, n_cols, subplot)
        ax.set_title(col + ' vs. ' + skills_ratings_non_gk_new[x])
        if x == i:
            sns.kdeplot(data=non_gk_skills_new, x=skills_ratings_non_gk_new[i], fill=True)#, hue='')
        else:
            sns.scatterplot(data=non_gk_skills_new.sample(5200), x=skills_ratings_non_gk_new[i], y=skills_ratings_non_gk_new[x], hue='kmeans_2_new', palette=['blue', 'red'])
    print('*  '*50)
    print(col)
    fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.show()   
    plt.close()

REVISAR CONCLUSION


Se observa que hay variables para las cuales los clusters quedan superpuestos, indicando que dichas variables no están aportando información para la agrupación que se analiza. Para algunas variables vistas de a pares, la división es bien clara, por ejemplo: `shooting` vs `passing`, `shotting` vs `defending`, `shotting` vs `attacking_short_passing`, `shotting` vs `mentality_interceptions`, `shotting` vs `defending_marking_awareness`, `shotting` vs `defending_standing_tackle`, `shotting` vs `defendindg_sliding_tackle`, `pacing` vs `defending`, `passing` vs `mentality_interceptions`, etc. Esto nuevamente refuerza la hipótesis de que para cada posición habrá habilidades en las que los distintos jugadores se irán destacando.

# CHICOS aca quise probar reagrupar las posiciones pero no me salió

In [ ]:
forwards=['RF', 'ST', 'LW', 'LF', 'RS', 'LS', 'RM', 'LM','RW']
midfielders=['RCM','LCM','LDM','CAM','CDM','LAM','RDM','CM','RAM','CF']
defenders=['RCB','CB','LCB','LB','RB','RWB','LWB']
goalkeepers=['GK']

def pos2(position):
    if position in forwards:
        return 'Forward'
    
    elif position in midfielders:
        return 'Midfielder'
    
    elif position in defenders:
        return 'Defender'
    
    elif position in goalkeepers:
        return 'GK'
    
    else:
        return 'nan'

In [ ]:
df["position_grouped"]=df["player_positions"].apply(lambda x: pos2(x))
df["position_grouped"].value_counts()

#### 3.2. MeanShift

En el punto anterior se probó Kmeans con 2, 3 y 4 clusters. La función MeanShift en cambio no requiere que sepamos de antemano cuántos clusters puede ser apropiado para el dataset sino que obtiene una cantidad de clusters.

In [ ]:
from numpy import random
ms = MeanShift(bandwidth=2, seeds=[random.randint(300, high=10000, size=37, dtype=int)])
ms.fit(non_gk_skills_new)
n_clusters = len(np.unique(ms.labels_))

non_gk_skills_new['ms_clusters'] = ms.labels_ # Agregamos las etiquetas al df

print("Cantidad de clusters encontrados por MeanShift : %d" % n_clusters)

#### 3.3. DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=10, min_samples=2).fit(non_gk_skills)
dbscan.labels_
#n_clusters_ = len(labels_unique)

In [ ]:
print('DBSCAN encontró ', max(dbscan.labels_)+1, 'clusters, según los hiperparámetros elegidos')

#### 3.3. Elección justificada de hiper-parámetros

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i).fit(non_gk_skills_new).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

## 4. Evaluación y Análisis de los clusters encontrados

## 5. ¿Se realizó alguna normalización o escalado de la base? ¿Por qué?

## 6. Uso de alguna transformación (proyección, Embedding) para visualizar los resultados y/o usarla como preprocesado para aplicar alguna técnica de clustering